In [180]:
import numpy as np
import pandas as pd
from transaction_dates import *
from imiona import *
from collections import Counter
from samochody import *
from czesci import *

# Pracownicy

In [181]:
birth_dates_lims = [1960, 1995]
begining_date = "2014-01-01"
end_date = "2018-01-01"

def rind(a, b):
    return np.random.randint(a, b)

jobs_list = np.array(["menadżer", "sprzedawca", "mechanik", "pomocnik mechanika", 
                       "diagnosta samochodowy", "sprzątający", "informatyk", "księgowy"])
jobs_amount = np.array([1, rind(2, 5), rind(3, 6), rind(1, 3),
                        rind(1, 3), rind(1, 3), rind(1, 3), rind(0, 2)])
fixed_jobs = np.array([0, 1, 2, np.sum(jobs_amount[:2]), np.sum(jobs_amount[0:6])]) # zaczynamy z szefem, dwoma sprzedawcami i mechanikiem

employment_dates = random_transaction_dates(begining_date, end_date, np.sum(jobs_amount))
employment_dates[fixed_jobs] = begining_date  

wages_lims = [(60, 75), (45, 60), (45, 60), (35, 45),
              (47, 60), (35, 40), (40, 60), (40, 53)]

employees_df = pd.DataFrame(columns=["id", "imię", "nazwisko", "PESEL",
                                     "miasto", "adres", "telefon", "email", "data_zatrudnienia",
                                     "data_zwolnienia", "stanowisko", "płaca"])
dismissal_dates = [[] for i in range(len(jobs_list))]
for (i, job) in enumerate(jobs_list):

    begin_times = ["" for j in range(jobs_amount[i])]
    for j in range(jobs_amount[i]):
        employment_date = employment_dates[np.sum(jobs_amount[:i]) + j]
        if np.random.rand() <= 0.5:
            first_name, last_name = random_name_man()
            pesel, birth_date = random_pesel_date_man(birth_dates_lims)
            sex = "M"
            city = random_city(prob=0.7)
            address = random_street()
            phone = random_phone()
            #email = random_email(first_name, last_name, birth_date[-4:])
            email = employee_email(first_name, last_name)
            employment_date = employment_dates[np.sum(jobs_amount[:i]) + j]
            wage = np.random.choice(np.arange(wages_lims[i][0], wages_lims[i][1], 0.1))
        else:
            first_name, last_name = random_name_woman()
            pesel, birth_date = random_pesel_date_woman(birth_dates_lims)
            sex = "K"
            city = random_city(prob=0.7)
            address = random_street()
            phone = random_phone()
            #email = random_email(first_name, last_name, birth_date[-4:])
            email = employee_email(first_name, last_name)
            employment_date = employment_dates[np.sum(jobs_amount[:i]) + j]
            wage = np.random.choice(np.arange(wages_lims[i][0], wages_lims[i][1], 0.1))
        begin_times[j] = employment_date

        employees_df.loc[np.sum(jobs_amount[:i]) + j] = [0, first_name, last_name, pesel,
                                                         city, address, phone, email, employment_date,
                                                         None, job, wage]
    dismissal_dates_j = [None for j in range(jobs_amount[i])]
    c = 0
    for j in range(len(begin_times)):
        if len(begin_times) >= 2:
            min_date = min(begin_times[:j] + begin_times[j+1:])
        else:
            min_date = begin_times[j]
        if np.random.rand() <= (jobs_amount[i]-1)/10 * 1/(1+c):
            dismissal_date = random_transaction_dates(str(max(min_date, begin_times[j])), end_date, 1)[0]
            okay = False
            for k in range(len(dismissal_dates_j)):
                if (dismissal_dates_j[k] is not None) and (dismissal_date >=  begin_times[k]) and (dismissal_date <= dismissal_dates_j[k]):
                    okay = True
                    break
                elif dismissal_date >=  begin_times[k]:
                    okay = True
                    break
            if okay:
                dismissal_dates_j[j] = dismissal_date
                c += 1
    dismissal_dates[i] = dismissal_dates_j

diss = []
for i in range(len(dismissal_dates)):
    diss += dismissal_dates[i]
employees_df["data_zwolnienia"] = diss

employees_df = employees_df.sort_values(by="data_zatrudnienia")   
employees_df["id"] =  np.arange(1, len(employees_df)+1)

In [182]:
employees_df

,id,imię,nazwisko,PESEL,miasto,adres,telefon,email,data_zatrudnienia,data_zwolnienia,stanowisko,płaca
0,1,Klaudia,Kabelis,65062102620,Wrocław,ul. Kolejowa 10/28,48925887344,klaudia.kabelis@kuba.pl,2014-01-01,None,menadżer,70.6
1,2,Olha,Danielewicz,74050959744,Wrocław,ul. Pogodna 97/11,48248312807,olha.danielewicz@kuba.pl,2014-01-01,None,sprzedawca,50.1
2,3,Emilia,Jusińska,88010965128,Wrocław,ul. Sportowa 23/44,48403181901,emilia.jusinska@kuba.pl,2014-01-01,None,sprzedawca,58.0
5,4,Anna,Kowalczyk,67071094543,Wrocław,ul. Miodowa 87,48786151166,anna.kowalczyk@kuba.pl,2014-01-01,2017-11-04,mechanik,54.6
14,5,Kostiantyn,Zakhozhyi,66090745759,Ostrowiec Świętokrzyski,ul. Gajowa 1,48000176135,kostiantyn.zakhozhyi@kuba.pl,2014-01-01,None,informatyk,58.8
3,6,Irena,Wołowska,71081617047,Wrocław,ul. Zieleni 79/1,48348982460,irena.wolowska@kuba.pl,2015-01-14,None,sprzedawca,46.5
4,7,Hannah,Kozelan,86110189046,Wrocław,ul. Śliwkowa 49,48580590907,hannah.kozelan@kuba.pl,2015-01-31,None,sprzedawca,51.7
6,8,Kamil,Modestowicz-Dobrowolski,64101372237,Wrocław,ul. Spadowa 24/12,48107174944,kamil.modestowicz-dobrowolski@kuba.pl,2015-07-25,2016-11-05,mechanik,56.0
7,9,Ewa,Wysowska,65102994028,Wrocław,ul. Daliowa 28/25,48208840145,ewa.wysowska@kuba.pl,2015-08-22,None,mechanik,48.8
8,10,Lucyna,Horniak,62121693707,Wrocław,ul. Zielna 51,48803989392,lucyna.horniak@kuba.pl,2016-01-10,None,mechanik,51.6


In [183]:
employees_df.to_csv("outputs/pracownicy.csv", index=False)

# Transakcje

In [184]:
no_transactions = np.random.poisson(2000)
begining_buy = 20

dates = random_transaction_dates(begining_date, end_date, no_transactions)

In [185]:
client_ids = np.zeros(len(dates), dtype=int) # array with client ids for table with transactions

j = 1
for i in range(len(dates)):
    prob = i/(2*len(dates))
    if np.random.rand() <= prob:
        client_ids[i] = np.random.choice(np.unique(client_ids[:i]))
    else:
        client_ids[i] = j
        j += 1

In [186]:
car_ids = np.zeros(len(dates), dtype=int) # array with car ids for table with transactions
car_ids[:begining_buy] = np.arange(1, 21)

client_ids = np.zeros(len(dates), dtype=int) # array with client ids for table with transactions
client_ids[:begining_buy] = np.arange(1, 21)
year_ceil = [int(date[:4])-19 for date in dates[:20]]

transactions_types = np.zeros(len(dates))

clients_cars = {}
our_cars = np.arange(1, 21).tolist()
not_fixed_our_cars = our_cars[:]

jp = 21
jc = 21
for i in range(20, len(dates)):
    weights = np.array([1.5*len(clients_cars)/max(1, len(our_cars)) , len(our_cars)*50, len(clients_cars)/(max(1, len(our_cars)))])
    weights = weights/np.sum(weights)
    transaction_type = np.random.choice(np.arange(3), p=weights) # 0-buy, 1-sell, 2-service
    transactions_types[i] = transaction_type
    if transaction_type == 0:
        prob = i/(2*len(dates)) 
        if np.random.rand() <= prob: #client that already is in database
            client_ids[i] = np.random.choice(np.unique(client_ids[:i]))
            if client_ids[i] in clients_cars and np.random.rand() <= 0.6 and len(clients_cars[client_ids[i]]) >= 1:
                car_ids[i] = np.random.choice(clients_cars[client_ids[i]])
                clients_cars[client_ids[i]].remove(car_ids[i])
            else:
                car_ids[i] = jc
                jc += 1
            our_cars.append(car_ids[i])
            not_fixed_our_cars.append(car_ids[i])
        else:
            client_ids[i] = jp
            car_ids[i] = jc
            our_cars.append(car_ids[i])
            not_fixed_our_cars.append(car_ids[i])
            jp += 1
            jc += 1
            year_ceil.append(int(dates[i][:4])-19)

    if transaction_type == 1:
        prob = i/(2*len(dates))
        if np.random.rand() <= prob: #client that already is in database
            client_ids[i] = np.random.choice(np.unique(client_ids[:i]))
            car_ids[i] = np.random.choice(our_cars)
            if client_ids[i] in clients_cars:
                clients_cars[client_ids[i]].append(car_ids[i])
            else:
                clients_cars[client_ids[i]] = [car_ids[i]]
            our_cars.remove(car_ids[i])
            if car_ids[i] in not_fixed_our_cars:
                not_fixed_our_cars.remove(car_ids[i])
        else:
            client_ids[i] = jp
            car_ids[i] = np.random.choice(our_cars)
            clients_cars[client_ids[i]] = [car_ids[i]]
            our_cars.remove(car_ids[i])
            if car_ids[i] in not_fixed_our_cars:
                not_fixed_our_cars.remove(car_ids[i])
            jp += 1
            year_ceil.append(int(dates[i][:4])-19)
    
    if transaction_type == 2:
        prob = i/(2*len(dates))
        if len(not_fixed_our_cars) >= 1 and np.random.rand() <= weights[1]: #fixing car we currently have
           client_ids[i] = 0
           car_ids[i] = np.random.choice(not_fixed_our_cars)
           not_fixed_our_cars.remove(car_ids[i])
        elif np.random.rand() <= prob: #client that already is in database
            client_ids[i] = np.random.choice(np.unique(client_ids[:i]))
            if client_ids[i] in clients_cars:
                if np.random.rand() <= 0.6 and len(clients_cars[client_ids[i]]) >= 1:
                    car_ids[i] = np.random.choice(clients_cars[client_ids[i]])
                else:
                    car_ids[i] = jc
                    clients_cars[client_ids[i]].append(car_ids[i])
                    jc += 1
            else:
                car_ids[i] = jc
                clients_cars[client_ids[i]] = [car_ids[i]]
                jc += 1
        else:
            client_ids[i] = jp
            car_ids[i] = jc
            clients_cars[client_ids[i]] = [car_ids[i]]
            jp += 1
            jc += 1
            year_ceil.append(int(dates[i][:4])-19)

In [187]:
len(year_ceil), len(pd.unique(client_ids))

(1369, 1370)

In [188]:
year_ceil

[1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,

In [189]:
buy_indexes = np.where(transactions_types == 0)
sell_indexes = np.where(transactions_types == 1)
service_indexes = np.where(transactions_types == 2)

In [190]:
len(buy_indexes[0]), len(sell_indexes[0]), len(service_indexes[0])

(783, 778, 419)

In [191]:
transaction_dates_buy = dates[buy_indexes]
transaction_dates_sell = dates[sell_indexes]
transaction_dates_service = dates[service_indexes]

client_ids_buy = client_ids[buy_indexes]
client_ids_sell = client_ids[sell_indexes]
client_ids_service = client_ids[service_indexes]

car_ids_buy = car_ids[buy_indexes]
car_ids_sell = car_ids[sell_indexes]
car_ids_service = car_ids[service_indexes]

In [192]:
buy_employee_id = np.zeros(len(transaction_dates_buy))

for i in range(len(transaction_dates_buy)):
    filter1 = employees_df["stanowisko"] == "sprzedawca"
    filter2 = employees_df["data_zatrudnienia"] <= transaction_dates_buy[i]
    filter3 = pd.notna(employees_df["data_zwolnienia"])
    filter4 = employees_df["data_zwolnienia"] < transaction_dates_buy[i]
    buy_employee_id[i] = np.random.choice(employees_df['id'][filter1 & filter2 & ~(filter3 & filter4)])

In [193]:
sell_employee_id = np.zeros(len(transaction_dates_sell))

for i in range(len(transaction_dates_sell)):
    filter1 = employees_df["stanowisko"] == "sprzedawca"
    filter2 = employees_df["data_zatrudnienia"] <= transaction_dates_sell[i]
    filter3 = pd.notna(employees_df["data_zwolnienia"])
    filter4 = employees_df["data_zwolnienia"] < transaction_dates_sell[i]
    sell_employee_id[i] = np.random.choice(employees_df['id'][filter1 & filter2 & ~(filter3 & filter4)])

In [194]:
service_employee_id = np.zeros(len(transaction_dates_service))

for i in range(len(transaction_dates_service)):
    filter1 = employees_df["stanowisko"] == "mechanik"
    filter2 = employees_df["data_zatrudnienia"] <= transaction_dates_service[i]
    filter3 = pd.notna(employees_df["data_zwolnienia"])
    filter4 = employees_df["data_zwolnienia"] < transaction_dates_service[i]
    service_employee_id[i] = np.random.choice(employees_df['id'][filter1 & filter2 & ~(filter3 & filter4)])


# klienci

In [195]:
year_floor = 1960
transaction_counter = Counter(client_ids)

clients_df = pd.DataFrame(columns=["id", "imię", "nazwisko", "płeć", "PESEL", "miasto", "adres", "karta_lojalnościowa", "telefon", "email"])

menager = employees_df.iloc[0]
if int(menager["PESEL"][-2]) % 2 == 1:
    menager_sex = "M"
else:
    menager_sex = "K"
clients_df.loc[0] = [0, menager["imię"], menager["nazwisko"], menager_sex, menager["PESEL"], menager["miasto"], menager["adres"], False, menager["telefon"], menager["email"]]

unique_clients = np.unique(client_ids)
unique_clients = unique_clients[unique_clients != 0]
for i in range(len(unique_clients)):
    if np.random.rand() <= 0.5: # chłop probability
        id_ = unique_clients[i]
        first_name, last_name = random_name_man()
        if id_ in client_ids_buy or id_ in client_ids_sell:
            pesel, birth_date = random_pesel_date_man([year_floor, year_ceil[i]])
            year = birth_date[-4:]
        else:
            pesel = None
            birth_date = None
            year = str(np.random.randint(year_floor, year_ceil[i]))
        sex = 'M'
        city = random_city()
        address = random_street()
        if transaction_counter[id_] >= 3:
            loyalty_card = True
        else:
            loyalty_card = False
        phone = random_phone()
        email = random_email(first_name, last_name, year)
        clients_df.loc[i+1] = [unique_clients[i], first_name, last_name, sex, pesel, city, address, loyalty_card, phone, email]
    else:
        id_ = unique_clients[i]
        first_name, last_name = random_name_woman()
        if id_ in client_ids_buy or id_ in client_ids_sell:
            pesel, birth_date = random_pesel_date_man([year_floor, year_ceil[i]])
            year = birth_date[-4:]
        else:
            pesel = None
            birth_date = None
            year = str(np.random.randint(year_floor, year_ceil[i]))
        sex = 'K'
        city = random_city()
        address = random_street()
        if transaction_counter[id_] >= 3:
            loyalty_card = True
        else:
            loyalty_card = False
        phone = random_phone()
        email = random_email(first_name, last_name, year)
        clients_df.loc[i+1] = [unique_clients[i], first_name, last_name, sex, pesel, city, address, loyalty_card, phone, email]
                             
clients_df.astype({"PESEL": str})

,id,imię,nazwisko,płeć,PESEL,miasto,adres,karta_lojalnościowa,telefon,email
0,0,Klaudia,Kabelis,K,65062102620,Wrocław,ul. Kolejowa 10/28,False,48925887344,klaudia.kabelis@kuba.pl
1,1,Mykola,Wlizło,M,70031058190,Wrocław,ul. Lwowska 86/35,True,48829038265,h7w38b59rju77l2o6@gmail.com
2,2,Teresa,Kocot,K,77092009510,Wrocław,ul. Otolanka 41/51,False,48697524943,kocot772@yahoo.com
3,3,Szymon,Fardulekas,M,89021860314,Brzeg,ul. św. Jana 49/23,False,48991330591,s-fardulekas@onet.pl
4,4,Adam,Pielka,M,91101880130,Wrocław,ul. Jana Lechonia 82/35,False,48198097912,ol0v3038lq808x@gmail.com
...,...,...,...,...,...,...,...,...,...,...
1365,1365,Dawid,Lisiewicz,M,80040521534,Wrocław,ul. Szmaragdowa 87,False,48270249119,lisiewicz1@interia.pl
1366,1366,Mirosław,Kaus,M,74122165899,Konin,ul. Wzgórz Trzebnickich 5/51,False,48573817626,m_kaus74@gmail.com
1367,1367,Adam,Krasnodębski,M,77022392736,Wrocław,ul. Zyndrama z Maszkowic 91/38,False,48653184906,ada_krasno9560@o2.pl
1368,1368,Ewelina,Szmurło,K,68062620996,Wrocław,ul. Kazimierza Wierzyńskiego 51/50,False,48184228385,szmurlo689@gmail.com


In [196]:
clients_with_lc = clients_df[clients_df["karta_lojalnościowa"]]["id"].values

loyalty_card_dates = [() for i in range(len(clients_with_lc))]
for (i, client) in enumerate(clients_with_lc):
    loyalty_card_dates[i] = client, dates[np.where(client_ids == client)[0][2]]


In [197]:
clients_df

,id,imię,nazwisko,płeć,PESEL,miasto,adres,karta_lojalnościowa,telefon,email
0,0,Klaudia,Kabelis,K,65062102620,Wrocław,ul. Kolejowa 10/28,False,48925887344,klaudia.kabelis@kuba.pl
1,1,Mykola,Wlizło,M,70031058190,Wrocław,ul. Lwowska 86/35,True,48829038265,h7w38b59rju77l2o6@gmail.com
2,2,Teresa,Kocot,K,77092009510,Wrocław,ul. Otolanka 41/51,False,48697524943,kocot772@yahoo.com
3,3,Szymon,Fardulekas,M,89021860314,Brzeg,ul. św. Jana 49/23,False,48991330591,s-fardulekas@onet.pl
4,4,Adam,Pielka,M,91101880130,Wrocław,ul. Jana Lechonia 82/35,False,48198097912,ol0v3038lq808x@gmail.com
...,...,...,...,...,...,...,...,...,...,...
1365,1365,Dawid,Lisiewicz,M,80040521534,Wrocław,ul. Szmaragdowa 87,False,48270249119,lisiewicz1@interia.pl
1366,1366,Mirosław,Kaus,M,74122165899,Konin,ul. Wzgórz Trzebnickich 5/51,False,48573817626,m_kaus74@gmail.com
1367,1367,Adam,Krasnodębski,M,77022392736,Wrocław,ul. Zyndrama z Maszkowic 91/38,False,48653184906,ada_krasno9560@o2.pl
1368,1368,Ewelina,Szmurło,K,68062620996,Wrocław,ul. Kazimierza Wierzyńskiego 51/50,False,48184228385,szmurlo689@gmail.com


In [198]:
clients_df.to_csv("outputs/klienci.csv", index=False)

In [199]:
np.sum(clients_df['karta_lojalnościowa'])

np.int64(75)

In [200]:
np.sum(pd.isna(clients_df["PESEL"]))

np.int64(152)

# auta

In [201]:
rng = np.random.default_rng()

data_car, data_bike = get_car_data()
make_list_car, model_list_car, car_weights = get_car_weights()
make_list_bike, model_list_bike, bike_weights = get_bike_weights()

In [202]:
def create_car(i):

    is_car = rng.random() < 0.9
    colors = np.array(('czarny', 'biały', 'szary', 'srebrny', 'niebieski', 'czerwony', 'brązowy', 'złoty', 'żółty', 'inny'))
    colors_weights = np.array((0.22, 0.19, 0.18, 0.15, 0.10, 0.09, 0.03, 0.02, 0.01, 0.01))

    if is_car:
        index = rng.choice(len(make_list_car), p=car_weights)
        make, model = make_list_car[index], model_list_car[index]
        weights = data_car[(data_car['make'] == make) & (data_car['model'] == model)]
        body = np.array(('SUV', 'Hatchback', 'Sedan', 'Kombi', 'Coupe', 'Van', 'Minivan', 'Kabriolet'))
        body_weights = np.array((0.25, 0.25, 0.23, 0.17, 0.06, 0.03, 0.009, 0.001))
    else:
        index = rng.choice(len(make_list_bike), p=bike_weights)
        make, model = make_list_bike[index], model_list_bike[index]
        weights = False
        body = False
        body_weights = False

    wheels = 4 if is_car else 2
    year = get_year()
    gear = get_gear(weights, is_car)
    fuel = get_fuel(weights, is_car)
    post_accident = get_post_accident()
    hp = get_hp(weights, is_car)
    color = get_color(colors, colors_weights)
    body_type = get_body_type(body, body_weights, is_car)
    engine_capacity = get_engine_capacity(hp, is_car)
    doors = get_doors(is_car)
    seats = get_num_of_seats(is_car)
    available = i+1 in our_cars
    
    return [i+1, make, model, wheels, year, body_type, post_accident, color, fuel, engine_capacity, hp, gear, doors, seats, available]


def create_car_table(n):
    
    df = pd.DataFrame(columns=['id', 'marka', 'model', 'liczba_kół', 'rok_produkcji', 'typ_nadwozia', 'czy_powypadkowy', 'kolor', 'typ_paliwa', 'pojemność_silnika', 'moc', 'skrzynia_biegów', 'liczba_drzwi', 'liczba_miejsc', 'czy_dostępny'])
    for i in range(n):
        row = create_car(i)
        df.loc[i] = row
    return df


In [203]:
df_samochody = create_car_table(max(car_ids))

In [204]:
df_samochody

,id,marka,model,liczba_kół,rok_produkcji,typ_nadwozia,czy_powypadkowy,kolor,typ_paliwa,pojemność_silnika,moc,skrzynia_biegów,liczba_drzwi,liczba_miejsc,czy_dostępny
0,1,Volvo,XC60,4,2007,SUV,False,biały,Benzyna,3.3,215,Automatyczna,5,5,False
1,2,Ford,Fiesta,4,2005,Kombi,False,brązowy,Benzyna,0.8,71,Manualna,5,2,False
2,3,Fiat,500X,4,2002,Kombi,False,czarny,Benzyna,1.8,110,Manualna,5,5,False
3,4,Volkswagen,Polo,4,2007,Kombi,False,czarny,Diesel,1.6,105,Manualna,5,5,False
4,5,Porsche,997,4,2004,Kombi,False,srebrny,Benzyna,8.0,500,Automatyczna,5,5,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
954,955,Nissan,Micra,4,2014,Kombi,False,szary,Benzyna,1.4,90,Manualna,5,2,True
955,956,Toyota,Auris,4,2006,SUV,False,czarny,Benzyna,2.1,132,Manualna,5,5,True
956,957,Audi,A5,4,2007,SUV,False,brązowy,Benzyna,2.4,150,Automatyczna,3,2,False
957,958,Volkswagen,Tiguan,4,2017,Sedan,False,srebrny,Diesel,2.2,140,Automatyczna,5,5,False


In [205]:
df_samochody.to_csv("outputs/samochody.csv", index=False)

In [206]:
pd.unique(df_samochody["skrzynia_biegów"])

array(['Automatyczna', 'Manualna', 'Półautomatyczna'], dtype=object)

# Usługi

In [207]:
service_employee = rng.choice(employees_df[employees_df['stanowisko'] == 'mechanik']['id'], len(car_ids_service))
service_ids = np.arange(len(transaction_dates_service)) + 1

df_services = pd.DataFrame()
df_services['id'] = service_ids
df_services['id_samochodu'] = car_ids_service
df_services['id_klienta'] = client_ids_service
df_services['id_pracownika'] = service_employee
df_services['data_zaksięgowania'] = transaction_dates_service

# Zakup samochodów

In [208]:
def car_purchase_price(hp, yr_bought, yr_prod, post_acc):
    return np.round(hp * 1000 / np.sqrt(int(yr_bought) - int(yr_prod) + 1) * (0.75 if post_acc else 1) * rng.uniform(0.8, 1.2), -2)

In [209]:
buy_employee = rng.choice(employees_df[employees_df['stanowisko'] == 'sprzedawca']['id'], len(car_ids_buy))
buy_ids = np.arange(len(transaction_dates_buy)) + 1

df_buy = pd.DataFrame()
df_buy['id'] = buy_ids
df_buy['id_samochodu'] = car_ids_buy
df_buy['id_klienta'] = client_ids_buy
df_buy['id_pracownika'] = buy_employee
df_buy['data_zaksięgowania'] = transaction_dates_buy

In [210]:

service_date_fix = pd.merge(df_samochody[['id', 'rok_produkcji']], df_services[['id_samochodu', 'data_zaksięgowania']], how='right', left_on='id', right_on='id_samochodu')
service_date_fix['data_zaksięgowania'] = service_date_fix['data_zaksięgowania'].str[:4]

buy_date_fix = pd.merge(df_samochody[['id', 'rok_produkcji']], df_buy[['id_samochodu', 'data_zaksięgowania']], how='right', left_on='id', right_on='id_samochodu')
buy_date_fix['data_zaksięgowania'] = buy_date_fix['data_zaksięgowania'].str[:4]

for i in range(len(buy_date_fix)):
    if str(buy_date_fix['rok_produkcji'].loc()[i]) >= str(buy_date_fix['data_zaksięgowania'].loc()[i]):
        car_index = buy_date_fix.loc[i, 'id_samochodu']
        df_samochody.loc[car_index-1, 'rok_produkcji'] = str(int(df_samochody.loc[car_index-1, 'rok_produkcji']) - 10)

for i in range(len(service_date_fix)):
    if str(service_date_fix['rok_produkcji'].loc()[i]) >= str(service_date_fix['data_zaksięgowania'].loc()[i]):
        car_index = service_date_fix.loc[i, 'id_samochodu']
        df_samochody.loc[car_index-1, 'rok_produkcji'] = str(int(df_samochody.loc[car_index-1, 'rok_produkcji']) - 10)

C:\Users\oskar.LAPTOP-KARLNF81\AppData\Local\Temp\ipykernel_14068\3526569416.py:10: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '2006' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_samochody.loc[car_index-1, 'rok_produkcji'] = str(int(df_samochody.loc[car_index-1, 'rok_produkcji']) - 10)


In [211]:
purchase_prices = []
for j, i in enumerate(car_ids_buy):
    row = df_samochody.loc()[i-1]
    year_bought = transaction_dates_buy[j][:4]
    purchase_prices.append(car_purchase_price(row['moc'], year_bought, row['rok_produkcji'], row['czy_powypadkowy']))

df_buy['cena'] = purchase_prices

In [212]:
df_buy

,id,id_samochodu,id_klienta,id_pracownika,data_zaksięgowania,cena
0,1,1,1,6,2014-01-02,74900.0
1,2,2,2,7,2014-01-03,25800.0
2,3,3,3,2,2014-01-04,33500.0
3,4,4,4,2,2014-01-07,39600.0
4,5,5,5,3,2014-01-07,154200.0
...,...,...,...,...,...,...
778,779,952,1363,7,2017-12-20,30600.0
779,780,954,1365,7,2017-12-27,21500.0
780,781,955,1366,7,2017-12-27,40000.0
781,782,956,1368,6,2017-12-28,30800.0


In [213]:
df_buy.to_csv("outputs/zakup_samochodu.csv", index=False)

# Sprzedaż samochodów

In [214]:
sell_employee = rng.choice(employees_df[employees_df['stanowisko'] == 'sprzedawca']['id'], len(car_ids_sell))
sell_ids = np.arange(len(transaction_dates_sell)) + 1

df_sell = pd.DataFrame()
df_sell['id'] = sell_ids
df_sell['id_samochodu'] = car_ids_sell
df_sell['id_klienta'] = client_ids_sell
df_sell['id_pracownika'] = sell_employee
df_sell['data_zaksięgowania'] = transaction_dates_sell

In [215]:
loyalty_card_customer_ids = [el[0] for el in loyalty_card_dates]
loyalty_card_dates_sep = [el[1] for el in loyalty_card_dates]

In [216]:
sell_prices = []
for i in car_ids_sell:
    b_price = df_buy[df_buy['id_samochodu'] == i]['cena'].values[0]
    sell_prices.append(np.round(b_price * rng.uniform(1.2, 1.5), -2))

for i in range(len(sell_ids)):
    customer_id = df_sell.loc()[i]['id_klienta']
    if clients_df[clients_df['id'] == customer_id]['karta_lojalnościowa'].bool() and customer_id in loyalty_card_customer_ids:
        date_index = loyalty_card_customer_ids.index(customer_id)
        if loyalty_card_dates_sep[date_index] < df_sell.loc()[i]['data_zaksięgowania']:
            sell_prices[i] *= 0.97
            #print(i)

sell_installments = []
for i in range(len(sell_ids)):
    if sell_prices[i] > 80000:
        inst_prob = [0.1, 0.2, 0.3, 0.4]
    else:
        inst_prob = [0.4, 0.3, 0.2, 0.1]
    installments = rng.choice([1, 6, 12, 18], p=inst_prob)
    if installments == 6:
        sell_prices[i] *= 1.02
    elif installments == 12:
        sell_prices[i] *= 1.04
    elif installments == 18:
        sell_prices[i] *= 1.06
    sell_installments.append(installments)
#print(sell_installments)

df_sell['cena'] = sell_prices
df_sell['liczba_rat'] = sell_installments
df_sell = df_sell[['id', 'id_samochodu', 'id_klienta', 'id_pracownika', 'cena', 'liczba_rat', 'data_zaksięgowania']]

C:\Users\oskar.LAPTOP-KARLNF81\AppData\Local\Temp\ipykernel_14068\1822833189.py:8: FutureWarning: Series.bool is now deprecated and will be removed in future version of pandas
  if clients_df[clients_df['id'] == customer_id]['karta_lojalnościowa'].bool() and customer_id in loyalty_card_customer_ids:


In [217]:
df_sell

,id,id_samochodu,id_klienta,id_pracownika,cena,liczba_rat,data_zaksięgowania
0,1,16,21,6,143948.0,18,2014-01-14
1,2,2,22,7,34424.0,12,2014-01-15
2,3,1,23,7,111300.0,1,2014-01-15
3,4,11,24,2,47328.0,6,2014-01-17
4,5,12,25,7,43700.0,1,2014-01-18
...,...,...,...,...,...,...,...
773,774,934,142,6,98050.0,18,2017-12-22
774,775,926,956,2,119674.0,18,2017-12-26
775,776,952,1367,6,37100.0,1,2017-12-28
776,777,951,873,2,54100.0,1,2017-12-28


In [218]:
np.sum(df_sell["cena"] == float('inf')) # wychodzą inf wiec trzeba poprawić

np.int64(0)

In [219]:
df_sell.to_csv("outputs/sprzedaż_samochodu.csv", index=False)

# Użyte części

In [220]:
data_parts = get_data_parts()
data_parts_filtered = data_parts[data_parts['motor'] == True]

def get_parts_indexes(car_id):
    if df_samochody.loc[car_id-1].values[3] == 2:
        parts_set = data_parts_filtered[['id','część']]
    else:
        parts_set = data_parts[['id','część']]
    
    parts_indexes = parts_set['id']
    num_parts = rng.choice([1, 2, 3], p=[0.6, 0.3, 0.1])
    indexes = rng.choice(parts_indexes, size=num_parts)
    return indexes

all_car_parts_indexes = [(i, get_parts_indexes(i)) for i in car_ids_service]
print(all_car_parts_indexes)

[(np.int64(24), array([48])), (np.int64(25), array([46, 31, 23])), (np.int64(31), array([35, 38, 18])), (np.int64(33), array([15,  5])), (np.int64(44), array([21])), (np.int64(45), array([14, 37])), (np.int64(46), array([34, 50])), (np.int64(47), array([50])), (np.int64(49), array([31])), (np.int64(50), array([20, 55])), (np.int64(52), array([42])), (np.int64(57), array([13, 21])), (np.int64(65), array([37, 41])), (np.int64(66), array([22, 48])), (np.int64(70), array([4])), (np.int64(71), array([30])), (np.int64(69), array([3])), (np.int64(72), array([21, 49])), (np.int64(73), array([41, 19,  0])), (np.int64(74), array([0])), (np.int64(75), array([15, 24])), (np.int64(78), array([38, 17])), (np.int64(79), array([33, 36])), (np.int64(84), array([9, 7])), (np.int64(87), array([10])), (np.int64(90), array([60])), (np.int64(92), array([20])), (np.int64(93), array([42])), (np.int64(94), array([15, 50])), (np.int64(96), array([30])), (np.int64(97), array([ 1, 49])), (np.int64(100), array([12

In [221]:
#df_samochody.loc()[1] - samochód o indeksie 0

def create_part(id, service_id, car):
    part = data_parts.loc()[id]
    if type(part['atrybuty']) != float:
        attributes = part['atrybuty'].split()
        for i in range(len(attributes)):
            if attributes[i] == 'marka':
                attributes[i] = car['marka']
            if attributes[i] == 'model':
                attributes[i] = car['model']
            if attributes[i] == 'kolor':
                attributes[i] = car['kolor']
            if attributes[i] == 'rozmiar':
                attributes[i] = 'R' + str(rng.choice([15, 16, 17]))
            if attributes[i] == 'material':
                attributes[i] = rng.choice(['skóra', 'materiał'], p=[0.2, 0.8])
            if attributes[i] == 'pojemnosc':
                attributes[i] = str(car['pojemność_silnika'])
            if attributes[i] == 'skrzynia':
                attributes[i] = str(car['skrzynia_biegów'])
        price = int(np.floor(part['cena']*car['pojemność_silnika']/1.8 * rng.uniform(0.8, 1.2)))
    else:
        attributes = ''
        price = part['cena']
    quantity = part['maxcount']
    return part['część'], ' '.join(attributes), price, service_id, quantity

parts = []
for service_id, element in enumerate(all_car_parts_indexes):
    car_id, parts_id = element
    car = df_samochody.loc()[car_id-1]
    for part_id in parts_id:
        parts.append(create_part(part_id, service_id+1, car))

parts = np.array(parts)
parts_df = pd.DataFrame(parts, columns=['nazwa', 'atrybuty', 'cena', 'id_usługi', 'ilość']).drop_duplicates(['nazwa', 'atrybuty'])
parts_df['id'] = np.arange(1, len(parts_df)+1)
parts_df = parts_df[['id', 'nazwa', 'atrybuty', 'cena', 'id_usługi', 'ilość']]

parts = pd.DataFrame(parts, columns=['nazwa', 'atrybut', 'cena', 'id_usługi', 'ilość'])

In [222]:
parts

,nazwa,atrybut,cena,id_usługi,ilość
0,Ogniwo paliwowe,Volkswagen Jetta,244,1,1
1,Tłumik,,200,2,1
2,Zagłówek,Fiat Punto Evo,13,2,2
3,Podświetlenie tablicy,,20,2,1
4,Przewód hamulcowy,,50,3,1
...,...,...,...,...,...
635,Światło mgłowe,Volkswagen Tiguan,68,416,1
636,Przewód hamulcowy,,50,417,1
637,Filtr paliwa,Nissan Micra,364,418,1
638,Głośnik,Nissan Micra,92,418,1


# Wyposażenie

In [223]:
eq_len = len(parts_df)
equipment_id = parts_df['id']
equipment_name = parts_df['nazwa']
equipment_attr = parts_df['atrybuty']
equipment_price = parts_df['cena']

df_equipment = pd.DataFrame()
df_equipment['id'] = equipment_id
df_equipment['nazwa'] = equipment_name
df_equipment['atrybut'] = equipment_attr
df_equipment['cena_części'] = equipment_price.astype(float)

In [224]:
df_equipment

,id,nazwa,atrybut,cena_części
0,1,Ogniwo paliwowe,Volkswagen Jetta,244.0
1,2,Tłumik,,200.0
2,3,Zagłówek,Fiat Punto Evo,13.0
3,4,Podświetlenie tablicy,,20.0
4,5,Przewód hamulcowy,,50.0
...,...,...,...,...
634,459,Akumulator,Volkswagen Tiguan,827.0
635,460,Światło mgłowe,Volkswagen Tiguan,68.0
637,461,Filtr paliwa,Nissan Micra,364.0
638,462,Głośnik,Nissan Micra,92.0


# Użyte części - uzupełnienie

In [225]:
used_parts_df = parts.merge(df_equipment, on=['nazwa', 'atrybut'])
used_parts_df.rename(columns={'id': 'id_części'}, inplace=True)
used_parts_df['ilość'] = pd.to_numeric(used_parts_df['ilość'])
ids = np.arange(len(used_parts_df))+1
used_parts_df['id'] = ids
used_parts_df = used_parts_df[['id', 'id_usługi', 'id_części', 'ilość', 'cena']]

In [226]:
used_parts_df

,id,id_usługi,id_części,ilość,cena
0,1,1,1,1,244
1,2,2,2,1,200
2,3,2,3,2,13
3,4,2,4,1,20
4,5,3,5,1,50
...,...,...,...,...,...
635,636,416,460,1,68
636,637,417,5,1,50
637,638,418,461,1,364
638,639,418,462,1,92


In [227]:
used_parts_df.to_csv("outputs/użyte_części.csv", index=False)

# Zakup części

In [228]:
# def get_service_date(part_id):
    
#     index_used_part = used_parts_df[used_parts_df['id_części'] == part_id].index[0]
#     service_date = df_services[df_services['id'] == index_used_part]['data_zaksięgowania']
#     return service_date
    

# buy_parts_quantity = used_parts_df.groupby(['id_części'])['ilość'].sum().reset_index()
# parts_purchase_pd = pd.DataFrame(columns=['id', 'id_części', 'koszt', 'ilość'])

# for i in range(len(buy_parts_quantity)):
#     part_id = buy_parts_quantity['id_części'][i]
#     quantity = buy_parts_quantity['ilość'][i]
#     price = df_equipment['cena_części'][df_equipment['id'] == part_id].to_numpy()
#     total_cost = np.floor(price * rng.uniform(0.75, 0.95))[0]
#     service_date = get_service_date(part_id)
    
#     row = [int(i+1), int(part_id), total_cost, int(quantity)]
#     parts_purchase_pd.loc[i] = row


In [229]:
# parts_purchase_pd

In [230]:
# parts_purchase_pd.to_csv("outputs/zakup_części.csv", index=False)

In [231]:
df_equipment = df_equipment[['id', 'nazwa', 'atrybut']]

In [232]:
df_equipment.to_csv("outputs/wyposażenie.csv", index=False)

# Usługi uzupełnienie

In [233]:
service_prices = np.zeros(len(service_ids))
for i in range(len(parts)):
    row = parts.loc()[i]
    service_index = int(row['id_usługi'])
    service_prices[service_index-1] += float(row['cena']) * int(row['ilość'])

df_services['cena'] = np.round(service_prices*rng.uniform(1.1, 1.4), -1).astype(int)

In [234]:
df_services.to_csv("outputs/usługi.csv", index=False)